In [1]:
# where is it ?

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [2]:
train_df_full = pd.read_csv('train.csv')#'/kaggle/input/playground-series-s4e9/train.csv')
test_df_full = pd.read_csv('test.csv')#'/kaggle/input/playground-series-s4e9/test.csv')

In [4]:

# preparing data, set tuning params
SEED = 2
DEV_FRACTION = 1
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)
N = 5
test_frac = 0.2

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

X['mileage'] = X.pop('milage')//100 # binning mileage

kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
scoring='neg_root_mean_squared_error'

model = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    RandomForestRegressor()
)

scores = -cross_val_score(model, X, y, scoring=scoring, cv=kfold, n_jobs=1)

print(F'CV RMSE score: {np.mean(scores):.5f} ± {np.std(scores):.5f}')

(47133, 13)
CV RMSE score: 84829.24372 ± 12046.91113


# producing the output

In [6]:
model = make_pipeline(
    TargetEncoder(random_state=0, target_type='continuous').set_output(transform='pandas'),
    StandardScaler().set_output(transform='pandas'),
    RandomForestRegressor()
)


# Step 2: Separate features (X) and target (y) from the full training set
X_train = train_df_full.drop(columns='price')
y_train = train_df_full['price']


# Step 3: Fit the model on the full training data
model.fit(X_train, y_train)

# Step 4: Prepare test data and make predictions
X_test = test_df_full#.drop(columns='id')  # Assuming 'id' column is in test data
predictions = model.predict(X_test)

# Step 5: Create a submission DataFrame
submission = pd.DataFrame({
    'id': test_df_full['id'],  # Ensure the 'id' from the test set is preserved
    'price': predictions       # The predicted prices
})

# Step 6: Save the submission to a CSV file
submission.to_csv('submission1_TargetRFScalePipeline.csv', index=False)  # Save as CSV